In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Menggunakan impor untuk memuat modul perpustakaan ke dalam memori program. 
import pandas as pd
# Membaca persona.csv sebagai panda DataFrame 
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/prak-apm2/file 3/potention.csv")

In [9]:
# Analisis statistik deskriptif
df.head() # menampilkan 5 baris pertama untuk memeriksa 

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,windows,male,bra,17
1,39,windows,male,bra,17
2,49,windows,male,bra,17
3,29,windows,male,tur,17
4,49,windows,male,tur,17


In [10]:
df.shape # menampilkan  dimensi DataFrame 

(5000, 5)

In [11]:
df.columns

Index(['PRICE', 'SOURCE', 'SEX', 'COUNTRY', 'AGE'], dtype='object')

In [12]:
df.describe().T # Menghasilkan statistik deskriptif 

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


In [13]:
df.isnull().values.any() # menampilkan sembarang nilai hilang di DataFrame 

False

In [14]:
df.isnull().sum() # Mengembalikan berapa banyak nilai yang hilang ada di DataFrame

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [16]:
# Jumlah unik <SOURCE>
df["SOURCE"].nunique() # Hitung jumlah elemen SOURCE yang berbeda 


2

In [18]:
df["SOURCE"].value_counts()# Mengembalikan jumlah baris SOURCE 

windows    2994
mac        2006
Name: SOURCE, dtype: int64

In [19]:
df["COUNTRY"].value_counts() # Mengembalikan jumlah baris COUNTRY

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [20]:
# Rincian negara dari rata-rata pendapatan
df.groupby("COUNTRY")["PRICE"].agg({"mean"})

,mean
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [21]:
# Negara dan Sumber rincian pendapatan rata-rata
df.groupby(["COUNTRY", 'SOURCE'])["PRICE"].mean()

COUNTRY  SOURCE 
bra      mac        34.299625
         windows    34.343035
can      mac        33.951456
         windows    33.330709
deu      mac        34.268817
         windows    33.869888
fra      mac        32.776224
         windows    34.312500
tur      mac        33.528302
         windows    35.903766
usa      mac        34.410628
         windows    33.737268
Name: PRICE, dtype: float64

In [22]:
# Pendapatan rata-rata berdasarkan variabel, 
agg_df = df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"])["PRICE"].mean().sort_values(ascending=False)
agg_df.head()

COUNTRY  SOURCE   SEX     AGE
bra      windows  male    46     59.0
usa      windows  male    36     59.0
fra      windows  female  24     59.0
usa      mac      male    32     54.0
fra      mac      male    20     49.0
Name: PRICE, dtype: float64

In [23]:
# Ubah nama indeks menjadi nama variabel
agg_df = agg_df.reset_index() # Jika Salah, kembalikan salinannya. Jika tidak, lakukan operasi di tempat dan kembalikan Tidak Ada.
agg_df.head() # menampilkan  nomor indeks baru

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,windows,male,46,59.0
1,usa,windows,male,36,59.0
2,fra,windows,female,24,59.0
3,usa,mac,male,32,54.0
4,fra,mac,male,20,49.0


In [24]:
# Ubah variabel AGE menjadi variabel kategori dan tambahkan ke agg_df
my_labels = ['0_18', '19_23', '24_30', '31_40', '41_70']
agg_df["AGE_CUT"] = pd.cut(x=agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=my_labels)
agg_df.tail(10) # menampilkan kembali data


,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CUT
339,bra,windows,male,23,21.5,19_23
340,tur,windows,male,21,19.0,19_23
341,bra,mac,male,47,19.0,41_70
342,tur,mac,male,47,19.0,41_70
343,bra,mac,female,34,19.0,31_40
344,usa,mac,female,30,19.0,24_30
345,usa,mac,female,38,19.0,31_40
346,fra,windows,male,18,19.0,0_18
347,can,windows,female,27,19.0,24_30
348,deu,windows,male,26,9.0,24_30


In [25]:
# Identifikasi pelanggan berbasis level (Persona) 
agg_df["customers_level_based"] = [f"{i[0]}_{i[1]}_{i[2]}_{i[-1]}" for i in agg_df.values]

In [26]:
agg_df = agg_df.loc[:, ["customers_level_based", "PRICE"]].groupby("customers_level_based").agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
agg_df["customers_level_based"].head() # menampilkan data kembali

0    fra_windows_female_24_30
1          tur_mac_male_24_30
2          tur_mac_male_31_40
3    tur_windows_female_31_40
4      can_windows_male_19_23
Name: customers_level_based, dtype: object

In [27]:
# Segmentasikan pelanggan baru (Persona)
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,fra_windows_female_24_30,45.428571,A
1,tur_mac_male_24_30,45.000000,A
2,tur_mac_male_31_40,42.333333,A
3,tur_windows_female_31_40,41.833333,A
4,can_windows_male_19_23,40.111111,A


In [28]:
# Jelaskan segmen dan terutama "C" 
agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["mean", "max", "sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.266588  32.333333   819.464474
C        33.455620  34.015873   903.301742
B        34.933929  35.972727   943.216084
A        38.580123  45.428571  1041.663328

In [29]:
agg_df[agg_df["SEGMENT"] == "C"].describe()

,PRICE
count,27.000000
mean,33.455620
std,0.498659
min,32.500000
25%,33.000000
50%,33.517264
75%,33.991748
max,34.015873


In [31]:
new_user = "fra_windows_male_24_30"
print(agg_df[agg_df["customers_level_based"] == new_user])

     customers_level_based  PRICE SEGMENT
72  fra_windows_male_24_30   33.0       C
